In [1]:
import os
import random
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
sys.path.append("..") # Adds higher directory to python modules path.
# import branchingdnn as branching
# dataset = branching.dataset.prepare.dataset(tf.keras.datasets.cifar10.load_data(),64,5000,22500,(227,227),include_targets=False)

In [2]:
seed = 66
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# tf.debugging.experimental.enable_dump_debug_info(logdir, tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# import csv
# with open('results/altTrain_labels.csv', newline='') as f:
    # reader = csv.reader(f,quoting=csv.QUOTE_NONNUMERIC)
    # alt_trainLabels = list(reader)
# with open('results/altTest_labels.csv', newline='') as f:
    # reader = csv.reader(f,quoting=csv.QUOTE_NONNUMERIC)
    # alt_testLabels = list(reader)

# altTraining = tf.data.Dataset.from_tensor_slices((train_images,alt_trainLabels))

# validation_images, validation_labels = train_images[:5000], alt_trainLabels[:5000]
# train_ds = tf.data.Dataset.from_tensor_slices((train_images, alt_trainLabels))
# test_ds = tf.data.Dataset.from_tensor_slices((test_images, alt_testLabels))
train_labels = tf.keras.utils.to_categorical(train_labels,10)
test_labels = tf.keras.utils.to_categorical(test_labels,10)

###normal method
validation_images, validation_labels = train_images[:5000], train_labels[:5000] #get the first 5k training samples as validation set
train_images, train_labels = train_images[5000:], train_labels[5000:] # now remove the validation set from the training set.
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

def augment_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    # image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (227,227))
    return image, label

train_ds_size = len(list(train_ds))
test_ds_size = len(list(test_ds))
validation_ds_size = len(list(validation_ds))

train_ds = (train_ds
                  .map(augment_images)
                  .shuffle(buffer_size=train_ds_size,seed=42,reshuffle_each_iteration=False)
                  .batch(batch_size=32, drop_remainder=True))

test_ds = (test_ds
                  .map(augment_images)
                #   .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=1, drop_remainder=True))

validation_ds = (validation_ds
                  .map(augment_images)
                #   .shuffle(buffer_size=validation_ds_size)
                  .batch(batch_size=32, drop_remainder=True))


In [2]:
# student_model = tf.keras.models.load_model("models/alexNetv6_logits_student.hdf5")

In [5]:
seed = 42
# random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
inputs = keras.Input(shape=(227,227,3))
# targets = keras.Input(shape=(10,))
x = keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3))(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
x = keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
# x = keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(x)
# x = keras.layers.BatchNormalization()(x)
# x = keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
# x = keras.layers.BatchNormalization()(x)
# x = keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
# x = keras.layers.BatchNormalization()(x)
# x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
# x = keras.layers.Flatten()(x)
# x = keras.layers.Dense(4096, activation='relu')(x)
# x = keras.layers.Dropout(0.5)(x)

# ### first branch
branchLayer = keras.layers.Flatten(name=tf.compat.v1.get_default_graph().unique_name("branch_flatten"))(x)
branchLayer = keras.layers.Dense(124, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch124"))(branchLayer)
branchLayer = keras.layers.Dense(64, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch64"))(branchLayer)
x = keras.layers.Dense(10, activation="softmax", name=tf.compat.v1.get_default_graph().unique_name("branch_output"))(branchLayer)


student_model = keras.Model(inputs=(inputs), outputs=[x], name="alexnet")

new_model = keras.Model(student_model.inputs, student_model.outputs, name=student_model.name)
new_model.summary()

# student_model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001,momentum=0.9), metrics=['accuracy'])
# student_model.save("models/alexNetv6_second_Exit.hdf5")

Model: "alexnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 conv2d_6 (Conv2D)           (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization_6 (Batc  (None, 55, 55, 96)       384       
 hNormalization)                                                 
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 27, 27, 96)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_7 (Batc  (None, 27, 27, 256)      1024

In [5]:
student_model.summary()

Model: "alexnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization (BatchN  (None, 55, 55, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_1 (Batc  (None, 27, 27, 256)      1024

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import copy
import itertools
import json
import os

import numpy as np
import six
from six.moves import zip  # pylint: disable=redefined-builtin

# from tensorflow.python.distribute import distribute_coordinator as dc
# from tensorflow.python.distribute import distribute_coordinator_context as dc_context
# from tensorflow.python.distribute import distribution_strategy_context as ds_context
# from tensorflow.python.distribute import parameter_server_strategy
# from tensorflow.python.distribute import values as ds_values
# from tensorflow.python.eager import backprop
# from tensorflow.python.eager import context
# from tensorflow.python.eager import def_function
# from tensorflow.python.eager import monitoring
# from tensorflow.python.framework import sparse_tensor
# from tensorflow.python.keras import callbacks as callbacks_module
# from tensorflow.python.keras import optimizers
# from tensorflow.python.keras.distribute import distributed_training_utils as dist_utils
# from tensorflow.python.keras.engine import compile_utils
from tensorflow.python.keras.engine import data_adapter
# from tensorflow.python.keras.engine import network
# from tensorflow.python.keras.engine import training_utils
# from tensorflow.python.keras.mixed_precision.experimental import loss_scale_optimizer as lso
# from tensorflow.python.keras.saving.saved_model import model_serialization
# from tensorflow.python.keras.utils import tf_utils
# from tensorflow.python.keras.utils import version_utils
# from tensorflow.python.keras.utils.mode_keys import ModeKeys
# from tensorflow.python.ops import array_ops
# from tensorflow.python.ops import sparse_ops
# from tensorflow.python.ops.ragged import ragged_concat_ops
# from tensorflow.python.ops.ragged import ragged_tensor
# from tensorflow.python.profiler import traceme
# from tensorflow.python.training.tracking import base as trackable
# from tensorflow.python.util import deprecation
# from tensorflow.python.util import nest
# from tensorflow.python.util import tf_decorator
# from tensorflow.python.util.tf_export import keras_export

from tensorflow.python.eager import context
from tensorflow.python.framework import composite_tensor
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import errors
from tensorflow.python.framework import errors_impl
from tensorflow.python.framework import func_graph
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.keras import backend
from tensorflow.python.keras.engine import base_layer
from tensorflow.python.keras.engine import base_layer_utils
from tensorflow.python.keras.engine import compile_utils
from tensorflow.python.keras.engine import input_layer as input_layer_module
from tensorflow.python.keras.engine import node as node_module
from tensorflow.python.keras.engine import training_utils
from tensorflow.python.keras.saving import hdf5_format
from tensorflow.python.keras.saving import save
from tensorflow.python.keras.saving.saved_model import network_serialization
from tensorflow.python.keras.utils import generic_utils
from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.keras.utils.io_utils import ask_to_proceed_with_overwrite
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops.ragged import ragged_tensor
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.training import checkpoint_management
from tensorflow.python.training import py_checkpoint_reader
from tensorflow.python.training.tracking import base as trackable
from tensorflow.python.training.tracking import data_structures
from tensorflow.python.training.tracking import layer_utils as trackable_layer_utils
from tensorflow.python.training.tracking import tracking
from tensorflow.python.training.tracking import util as trackable_utils
from tensorflow.python.util import nest
from tensorflow.python.util import serialization
from tensorflow.python.util import tf_inspect


# pylint: disable=g-import-not-at-top
try:
  import h5py
except ImportError:
  h5py = None

try:
  import yaml
except ImportError:
  yaml = None
# pylint: enable=g-import-not-at-top

In [3]:
class branched(keras.Model):
    def __init__(self, student, teacher):
        super(branched, self).__init__()
        self.teacher = teacher
        self.student = tf.keras.models.clone_model(student)
    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=1,        
    ):
        """ Configure the distiller.
        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(branched, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data
        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                (teacher_predictions / self.temperature),
                (student_predictions / self.temperature),
            )
            student_loss = student_loss * self.alpha
            distillation_loss = (distillation_loss) * (1 - self.alpha)
#             loss = distillation_loss
            loss=student_loss +distillation_loss
        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"loss":loss,"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        """The logic for one evaluation step.

        This method can be overridden to support custom evaluation logic.
        This method is called by `Model.make_test_function`.

        This function should contain the mathemetical logic for one step of
        evaluation.
        This typically includes the forward pass, loss calculation, and metrics
        updates.

        Configuration details for *how* this logic is run (e.g. `tf.function` and
        `tf.distribute.Strategy` settings), should be left to
        `Model.make_test_function`, which can also be overridden.

        Arguments:
          data: A nested structure of `Tensor`s.

        Returns:
          A `dict` containing values that will be passed to
          `tf.keras.callbacks.CallbackList.on_train_batch_end`. Typically, the
          values of the `Model`'s metrics are returned.
        """
        data = data_adapter.expand_1d(data)
        x, y, sample_weight = data_adapter.unpack_x_y_sample_weight(data)

        y_pred = self(x, training=False)
        print(self)
        # Updates stateful loss metrics.
        self.compiled_loss(
            y, y_pred, sample_weight, regularization_losses=self.losses)

        self.compiled_metrics.update_state(y, y_pred, sample_weight)
        return {m.name: m.result() for m in self.metrics}


    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

NameError: name 'keras' is not defined

In [25]:
class thing():
    def __call__(self, input):
        return input *3
    def call(self,input):
        return input * 2
t = thing()
t(10)

30

In [21]:
x = {"test":10}

y = x.get("test",0)
print(y)

10


In [3]:

import collections
import copy
import itertools
import warnings

from tensorflow.python.eager import context
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.keras import backend
from tensorflow.python.keras.engine import base_layer
from tensorflow.python.keras.engine import base_layer_utils
from tensorflow.python.keras.engine import input_layer as input_layer_module
from tensorflow.python.keras.engine import input_spec
from tensorflow.python.keras.engine import node as node_module
from tensorflow.python.keras.engine import training as training_lib
from tensorflow.python.keras.engine import training_utils
from tensorflow.python.keras.saving.saved_model import network_serialization
from tensorflow.python.keras.utils import generic_utils
from tensorflow.python.keras.utils import tf_inspect
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.training.tracking import base as trackable
from tensorflow.python.util import nest
from tensorflow.tools.docs import doc_controls


from tensorflow.python.keras.engine import functional

import types 

def _map_graph_network(inputs, outputs):
        """Validates a network's topology and gather its layers and nodes.

        Args:
        inputs: List of input tensors.
        outputs: List of outputs tensors.

        Returns:
        A tuple `(nodes, nodes_by_depth, layers, layers_by_depth)`.
        - nodes: list of Node instances.
        - nodes_by_depth: dict mapping ints (depth) to lists of node instances.
        - layers: list of Layer instances.
        - layers_by_depth: dict mapping ints (depth) to lists of layer instances.

        Raises:
        ValueError: In case the network is not valid (e.g. disconnected graph).
        """

        print("-----graph------")
        # "depth" is number of layers between output Node and the Node.
        # Nodes are ordered from inputs -> outputs.
        print("test")
        nodes_in_decreasing_depth, layer_indices = functional._build_map(outputs)
        network_nodes = {
          functional._make_node_key(node.layer.name, node.layer._inbound_nodes.index(node))
          for node in nodes_in_decreasing_depth
        }

        nodes_depths = {}  # dict {node: depth value}
        layers_depths = {}  # dict {layer: depth value}

        for node in reversed(nodes_in_decreasing_depth):
            print(node.layer.name)
            # If the depth is not set, the node has no outbound nodes (depth 0).
            depth = nodes_depths.setdefault(node, 0)

            # Update the depth of the corresponding layer
            previous_depth = layers_depths.get(node.layer, 0)
            # If we've seen this layer before at a higher depth,
            # we should use that depth instead of the node depth.
            # This is necessary for shared layers that have inputs at different
            # depth levels in the graph.
            depth = max(depth, previous_depth)
            layers_depths[node.layer] = depth
            nodes_depths[node] = depth

            # Update the depth of inbound nodes.
            # The "depth" of a node is the max of the depths
            # of all nodes it is connected to + 1.
            for node_dep in node.parent_nodes:
                previous_depth = nodes_depths.get(node_dep, 0)
                nodes_depths[node_dep] = max(depth + 1, previous_depth)

        # Handle inputs that are not connected to outputs.
        # We do not error out here because the inputs may be used to compute losses
        # and metrics.
        for input_t in inputs:
            input_layer = input_t._keras_history[0]
            if input_layer not in layers_depths:
                layers_depths[input_layer] = 0
                layer_indices[input_layer] = -1
                nodes_depths[input_layer._inbound_nodes[0]] = 0
                network_nodes.add(_make_node_key(input_layer.name, 0))

        # Build a dict {depth: list of nodes with this depth}
        nodes_by_depth = collections.defaultdict(list)
        for node, depth in nodes_depths.items():
            nodes_by_depth[depth].append(node)

        # Build a dict {depth: list of layers with this depth}
        layers_by_depth = collections.defaultdict(list)
        for layer, depth in layers_depths.items():
            layers_by_depth[depth].append(layer)

        # Get sorted list of layer depths.
        depth_keys = list(layers_by_depth.keys())
        depth_keys.sort(reverse=True)

        # Set self.layers ordered by depth.
        layers = []
        for depth in depth_keys:
            layers_for_depth = layers_by_depth[depth]
            # Network.layers needs to have a deterministic order:
            # here we order them by traversal order.
            layers_for_depth.sort(key=lambda x: layer_indices[x])
            layers.extend(layers_for_depth)

        # Get sorted list of node depths.
        depth_keys = list(nodes_by_depth.keys())
        depth_keys.sort(reverse=True)

        # Check that all tensors required are computable.
        # computable_tensors: all tensors in the graph
        # that can be computed from the inputs provided.
        computable_tensors = set()
        for x in inputs:
            computable_tensors.add(id(x))

        layers_with_complete_input = []  # To provide a better error msg.
        for depth in depth_keys:
            for node in nodes_by_depth[depth]:
                layer = node.layer
                if layer and not node.is_input:
                    for x in nest.flatten(node.keras_inputs):
                        if id(x) not in computable_tensors:
                            raise ValueError('Graph disconnected: '
                                             'cannot obtain value for tensor ' + str(x) +
                                             ' at layer "' + layer.name + '". '
                                             'The following previous layers '
                                             'were accessed without issue: ' +
                                             str(layers_with_complete_input))
                    for x in nest.flatten(node.outputs):
                        computable_tensors.add(id(x))
                    layers_with_complete_input.append(layer.name)

        # Ensure name unicity, which will be crucial for serialization
        # (since serialized nodes refer to layers by their name).
        all_names = [layer.name for layer in layers]
        for name in all_names:
            if all_names.count(name) != 1:
                raise ValueError('The name "' + name + '" is used ' +
                               str(all_names.count(name)) + ' times in the model. '
                               'All layer names should be unique.')
        print("end graph")

        return network_nodes, nodes_by_depth, layers, layers_by_depth
    
class BranchFinished():
    def __init__(self, val):
        self.val = val

class MyModel(tf.keras.Model):
    
#     def __init__(self, *args, **kwargs):
#         super(MyModel, self).__init__(*args, **kwargs)
#         print("---init----")
#         print(self._nodes_by_depth)
# #         self.self._nodes_by_depth = self.build_new_depth_map()
# #         print("---new depth ----")
# #         print(self._nodes_by_depth)
#         print("---end----")
#         funcType = types.MethodType
#         self._map_graph_network = funcType(_map_graph_network_new,self)
#         _init_graph_network_new(self, self.inputs,self.outputs)
#         super(MyModel, self).__init__(*args, **kwargs)
        
#     def build_new_depth_map(self):
#         results = _map_graph_network_new(self.inputs,self.outputs)
#         print("results", results)

    

    # @trackable.no_automatic_dependency_tracking
    def _init_graph_network(self, inputs, outputs):
        # This method is needed for Sequential to reinitialize graph network when
        # layer is added or removed.
        self._is_graph_network = True

        # Normalize and set self.inputs, self.outputs.
        if isinstance(inputs, list) and len(nest.flatten(inputs)) == 1:
          inputs = inputs[0]
        if isinstance(outputs, list) and len(nest.flatten(outputs)) == 1:
          outputs = outputs[0]
        self._nested_inputs = inputs
        self._nested_outputs = outputs
        self.inputs = nest.flatten(inputs)
        self.outputs = nest.flatten(outputs)

        # Models constructed with a single Tensor or list of Tensors can
        # be called with a dict, where the keys of the dict are the names
        # of the `Input` objects. Extra keys are ignored with warning.
        if not nest.is_nested(self._nested_inputs):
          self._enable_dict_to_input_mapping = True
        elif (isinstance(self._nested_inputs, (list, tuple)) and
              not any(nest.is_nested(t) for t in self._nested_inputs)):
          self._enable_dict_to_input_mapping = True
        elif (isinstance(self._nested_inputs, dict) and
              not any(nest.is_nested(t) for t in self._nested_inputs.values())):
          self._enable_dict_to_input_mapping = True
        else:
          self._enable_dict_to_input_mapping = False

        if not ops.executing_eagerly_outside_functions():
          if any(not hasattr(tensor, '_keras_history') for tensor in self.outputs):
            base_layer_utils.create_keras_history(self._nested_outputs)

        self._validate_graph_inputs_and_outputs()

        # A Network does not create weights of its own, thus it is already
        # built.
        self.built = True
        self._build_input_shape = nest.map_structure(lambda x: x.shape, inputs)
        self._compute_output_and_mask_jointly = True
        # `_expects_training_arg` is True since the `training` argument is always
        # present in the signature of the `call` method of a graph network.
        self._expects_training_arg = True
        self._expects_mask_arg = True
        # A graph network does not autocast inputs, as its layers will cast them
        # instead.
        self._autocast = False

        self._input_layers = []
        self._output_layers = []
        self._input_coordinates = []
        self._output_coordinates = []

        # This is for performance optimization when calling the Network on new
        # inputs. Every time the Network is called on a set on input tensors,
        # we compute the output tensors, output masks and output shapes in one pass,
        # then cache them here. When any of these outputs is queried later, we
        # retrieve it from there instead of recomputing it.
        self._output_mask_cache = {}
        self._output_tensor_cache = {}
        self._output_shape_cache = {}

        # Build self._output_layers:
        for x in self.outputs:
          layer, node_index, tensor_index = x._keras_history  # pylint: disable=protected-access
          self._output_layers.append(layer)
          self._output_coordinates.append((layer, node_index, tensor_index))

        # Build self._input_layers:
        for x in self.inputs:
          layer, node_index, tensor_index = x._keras_history  # pylint: disable=protected-access
          # It's supposed to be an input layer, so only one node
          # and one tensor output.
          assert node_index == 0
          assert tensor_index == 0
          self._input_layers.append(layer)
          self._input_coordinates.append((layer, node_index, tensor_index))

        # Keep track of the network's nodes and layers.
        nodes, nodes_by_depth, layers, _ = _map_graph_network(
            self.inputs, self.outputs)
        self._network_nodes = nodes
        self._nodes_by_depth = nodes_by_depth
        self._self_tracked_trackables = layers
        self._layer_call_argspecs = {}
        for layer in self._self_tracked_trackables:
          self._layer_call_argspecs[layer] = tf_inspect.getfullargspec(layer.call)

        # Build self.input_names and self.output_names.
        self._set_output_names()
        self.input_names = []
        self._feed_input_names = []
        self._feed_inputs = []
        self._feed_input_shapes = []
        for layer in self._input_layers:
          self.input_names.append(layer.name)
          if layer.is_placeholder:
            self._feed_input_names.append(layer.name)
            # Use batch_input_shape here because non-eager composite tensors may not
            # have a shape attribute that's meaningful (sparse, for instance, has
            # a tensor that's non-constant and needs to be fed). This means that
            # input layers that create placeholders will need to have the
            # batch_input_shape attr to allow for input shape validation.
            self._feed_input_shapes.append(layer._batch_input_shape)
            self._feed_inputs.append(layer.input)

        self._compute_tensor_usage_count()
        self._set_save_spec(self._nested_inputs)
        tf_utils.assert_no_legacy_layers(self.layers)    
        
#     def call(self, inputs, training=None, mask=None):
#         """Calls the model on new inputs.

#         In this case `call` just reapplies
#         all ops in the graph to the new inputs
#         (e.g. build a new computational graph from the provided inputs).

#         Arguments:
#             inputs: A tensor or list of tensors.
#             training: Boolean or boolean scalar tensor, indicating whether to run
#               the `Network` in training mode or inference mode.
#             mask: A mask or list of masks. A mask can be
#                 either a tensor or None (no mask).

#         Returns:
#             A tensor if there is a single output, or
#             a list of tensors if there are more than one outputs.
#         """
#         if not self._is_graph_network:
#             raise NotImplementedError('When subclassing the `Model` class, you should'
#                                     ' implement a `call` method.')

#         return self._run_internal_graph(
#             inputs, training=training, mask=mask,
#             convert_kwargs_to_constants=base_layer_utils.call_context().saving)
    
    
    ##### --------------------------------

#     def test_step(self, data):
#         """The logic for one evaluation step.

#         This method can be overridden to support custom evaluation logic.
#         This method is called by `Model.make_test_function`.

#         This function should contain the mathemetical logic for one step of
#         evaluation.
#         This typically includes the forward pass, loss calculation, and metrics
#         updates.

#         Configuration details for *how* this logic is run (e.g. `tf.function` and
#         `tf.distribute.Strategy` settings), should be left to
#         `Model.make_test_function`, which can also be overridden.

#         Arguments:
#           data: A nested structure of `Tensor`s.

#         Returns:
#           A `dict` containing values that will be passed to
#           `tf.keras.callbacks.CallbackList.on_train_batch_end`. Typically, the
#           values of the `Model`'s metrics are returned.
#         """
#         print(data)
#         data = data_adapter.expand_1d(data)
#         x, y, sample_weight = data_adapter.unpack_x_y_sample_weight(data)

#         y_pred = self(x, training=False)
#         print(self.layers)
#         # Updates stateful loss metrics.
#         self.compiled_loss(
#             y, y_pred, sample_weight, regularization_losses=self.losses)

#         self.compiled_metrics.update_state(y, y_pred, sample_weight)
#         return {m.name: m.result() for m in self.metrics}

    
#     def predict_step(self, data):
#         """The logic for one inference step.

#         This method can be overridden to support custom inference logic.
#         This method is called by `Model.make_predict_function`.

#         This method should contain the mathemetical logic for one step of inference.
#         This typically includes the forward pass.

#         Configuration details for *how* this logic is run (e.g. `tf.function` and
#         `tf.distribute.Strategy` settings), should be left to
#         `Model.make_predict_function`, which can also be overridden.

#         Arguments:
#           data: A nested structure of `Tensor`s.

#         Returns:
#           The result of one inference step, typically the output of calling the
#           `Model` on data.
#         """
#         data = data_adapter.expand_1d(data)
#         x, _, _ = data_adapter.unpack_x_y_sample_weight(data)
#         print("predict",self.layers)
#         try:
#             while True:
#                 result = self(x, training=False)
#         except BranchFinished as Branch_result:
#             print("branchPoint Hit")
#             print(Branch_result.val)
#             pass
#     # or print "Goodbye." if you want 
        
#         return result

    def turnOnThresholds(self):
        for layers in self.layers:
            if hasattr(layers,'threshold'):
                layers.threshold = True
    
class MyDenseLayer(tf.keras.layers.Layer):
    
    def __init__(self, nodes, name=None):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = nodes
        self.threshold = False
        
        
    def get_config(self):
            config = super().get_config().copy()
            config.update({
                'num_outputs': self.num_outputs,
                'name': self.name
            })
            return config
    
    def build(self, input_shape):
        print("="*20)
        print('we are here in build')
        self.kernel = self.add_weight("kernel",shape=      [int(input_shape[-1]),self.num_outputs],dtype=tf.float32)        
    
    def call(self, input):
#         print('='*20)
#         print('we are here in call')
#         print(input)
#         if self.threshold:
#             print('thresholds')
#             raise BranchFinished(tf.matmul(input, self.kernel))
            
        return tf.matmul(input, self.kernel)
    


In [3]:
inputs = keras.Input(shape=(227,227,3))
# targets = keras.Input(shape=(10,))
x = keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3))(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
x = keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
# x = keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(x)
# x = keras.layers.BatchNormalization()(x)
# x = keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
# x = keras.layers.BatchNormalization()(x)
# x = keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
# x = keras.layers.BatchNormalization()(x)
# x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
# x = keras.layers.Flatten()(x)
# x = keras.layers.Dense(4096, activation='relu')(x)
# x = keras.layers.Dropout(0.5)(x)

# ### first branch
branchLayer = keras.layers.Flatten(name=tf.compat.v1.get_default_graph().unique_name("branch_flatten"))(x)
dense1 = keras.layers.Dense(124, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("before"))(branchLayer)
exit1 =MyDenseLayer(10,name=tf.compat.v1.get_default_graph().unique_name("exit"))(dense1)
branchLayer = keras.layers.Dense(124, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("alternate1_"))(branchLayer)

branchLayer = keras.layers.Dense(64, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("alternate2_"))(branchLayer)
x = keras.layers.Dense(10, activation="softmax", name=tf.compat.v1.get_default_graph().unique_name("alternate_exit"))(branchLayer)


# model = keras.Model(inputs=(inputs), outputs=[x,exit1], name="alexnet")
model = MyModel(inputs=(inputs), outputs=[x,exit1], name="alexnet")

model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001,momentum=0.9), metrics=['accuracy'])
# student_model.save("models/alexNetv6_second_Exit.hdf5")
model.summary()

NameError: name 'MyDenseLayer' is not defined

In [ ]:
# model.turnOnThresholds()
result = model.predict(test_ds.take(1))
print(result)
predictions = []
for i in result:
    predictions.append(np.argmax(i,axis=1))
print(predictions)

In [ ]:
# NEXT THING: Figure out why the depth is messed up, and think of a way of reordering the graph so that the branches are evaluated before the main path continues. 

# also the whole thing doesn't work when the exits are re-ordered. and its all a bit of a hack really. I need a more elegant solution, otherwise I'll have to reupdate this every time this function changes... which might have to be the case.

In [63]:
import collections
import copy
import itertools
import warnings

from tensorflow.python.eager import context
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.keras import backend
from tensorflow.python.keras.engine import base_layer
from tensorflow.python.keras.engine import base_layer_utils
from tensorflow.python.keras.engine import input_layer as input_layer_module
from tensorflow.python.keras.engine import input_spec
from tensorflow.python.keras.engine import node as node_module
from tensorflow.python.keras.engine import training as training_lib
from tensorflow.python.keras.engine import training_utils
from tensorflow.python.keras.saving.saved_model import network_serialization
from tensorflow.python.keras.utils import generic_utils
from tensorflow.python.keras.utils import tf_inspect
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.training.tracking import base as trackable
from tensorflow.python.util import nest
from tensorflow.tools.docs import doc_controls
from tensorflow.python.keras.engine import functional

class simpleModel(tf.keras.Model):      
    

    def _map_graph_network(self, inputs, outputs,reverse =True):
      """Validates a network's topology and gather its layers and nodes.
    
      Args:
        inputs: List of input tensors.
        outputs: List of outputs tensors.

      Returns:
        A tuple `(nodes, nodes_by_depth, layers, layers_by_depth)`.
        - nodes: list of Node instances.
        - nodes_by_depth: dict mapping ints (depth) to lists of node instances.
        - layers: list of Layer instances.
        - layers_by_depth: dict mapping ints (depth) to lists of layer instances.

      Raises:
        ValueError: In case the network is not valid (e.g. disconnected graph).
      """
      print("_map_graph_network")
      # "depth" is number of layers between output Node and the Node.
      # Nodes are ordered from inputs -> outputs.
      nodes_in_decreasing_depth, layer_indices = functional._build_map(outputs)
      network_nodes = {
          functional._make_node_key(node.layer.name, node.layer._inbound_nodes.index(node))
          for node in nodes_in_decreasing_depth
      }

      nodes_depths = {}  # dict {node: depth value}
      layers_depths = {}  # dict {layer: depth value}
      if reverse:
          for node in reversed(nodes_in_decreasing_depth):
            # print(node.layer.name)
            # If the depth is not set, the node has no outbound nodes (depth 0).

            depth = nodes_depths.setdefault(node, 0)
            
            # Update the depth of the corresponding layer
            previous_depth = layers_depths.get(node.layer, 0)
            # If we've seen this layer before at a higher depth,
            # we should use that depth instead of the node depth.
            # This is necessary for shared layers that have inputs at different
            # depth levels in the graph.
            depth = max(depth, previous_depth)
            layers_depths[node.layer] = depth
            nodes_depths[node] = depth

            # Update the depth of inbound nodes.
            # The "depth" of a node is the max of the depths
            # of all nodes it is connected to + 1.
            for node_dep in node.parent_nodes:
              previous_depth = nodes_depths.get(node_dep, 0)
              nodes_depths[node_dep] = max(depth + 1, previous_depth)
            print(node.layer.name, " depth: ", depth )
          print("")
          for node in (nodes_in_decreasing_depth):
            # print(node.layer.name)
            # If the depth is not set, the node has no outbound nodes (depth 0).

            depth = nodes_depths.setdefault(node, 0)

            # Update the depth of the corresponding layer
            previous_depth = layers_depths.get(node.layer, 0)
            # print("prev depth", previous_depth)
            # If we've seen this layer before at a higher depth,
            # we should use that depth instead of the node depth.
            # This is necessary for shared layers that have inputs at different
            # depth levels in the graph.
            # if depth is not 0:
                # print(depth)
            depth = max(depth, previous_depth)
            layers_depths[node.layer] = depth
            nodes_depths[node] = depth

            # Update the depth of inbound nodes.
            # The "depth" of a node is the max of the depths
            # of all nodes it is connected to + 1.
            for node_dep in node.parent_nodes:
              previous_depth = nodes_depths.get(node_dep, 0)
              depth  = max(previous_depth - 1, 0)
              print(node.layer.name, ": ", depth, " ", node_dep.layer.name, ": ", previous_depth)
              
              # depth = previous_depth + 1
              nodes_depths[node] = depth
      # Handle inputs that are not connected to outputs.
      # We do not error out here because the inputs may be used to compute losses
      # and metrics.
      for input_t in inputs:
        input_layer = input_t._keras_history[0]
        if input_layer not in layers_depths:
          layers_depths[input_layer] = 0
          layer_indices[input_layer] = -1
          nodes_depths[input_layer._inbound_nodes[0]] = 0
          network_nodes.add(_make_node_key(input_layer.name, 0))

      # Build a dict {depth: list of nodes with this depth}
      nodes_by_depth = collections.defaultdict(list)
      for node, depth in nodes_depths.items():
        nodes_by_depth[depth].append(node)

      # Build a dict {depth: list of layers with this depth}
      layers_by_depth = collections.defaultdict(list)
      for layer, depth in layers_depths.items():
        layers_by_depth[depth].append(layer)

      # Get sorted list of layer depths.
      depth_keys = list(layers_by_depth.keys())
      depth_keys.sort(reverse=True)
      print(depth_keys)
      # Set self.layers ordered by depth.
      layers = []
      for depth in depth_keys:
        layers_for_depth = layers_by_depth[depth]
        # Network.layers needs to have a deterministic order:
        # here we order them by traversal order.
        layers_for_depth.sort(key=lambda x: layer_indices[x])
        layers.extend(layers_for_depth)

      # Get sorted list of node depths.
      depth_keys = list(nodes_by_depth.keys())
      depth_keys.sort(reverse=True)

      # Check that all tensors required are computable.
      # computable_tensors: all tensors in the graph
      # that can be computed from the inputs provided.
      computable_tensors = set()
      for x in inputs:
        computable_tensors.add(id(x))

      layers_with_complete_input = []  # To provide a better error msg.
      for depth in depth_keys:
        for node in nodes_by_depth[depth]:
          layer = node.layer
          if layer and not node.is_input:
            for x in nest.flatten(node.keras_inputs):
              if id(x) not in computable_tensors:
                raise ValueError('Graph disconnected: '
                                 'cannot obtain value for tensor ' + str(x) +
                                 ' at layer "' + layer.name + '". '
                                 'The following previous layers '
                                 'were accessed without issue: ' +
                                 str(layers_with_complete_input))
            for x in nest.flatten(node.outputs):
              computable_tensors.add(id(x))
            layers_with_complete_input.append(layer.name)

      # Ensure name unicity, which will be crucial for serialization
      # (since serialized nodes refer to layers by their name).
      all_names = [layer.name for layer in layers]
      for name in all_names:
        if all_names.count(name) != 1:
          raise ValueError('The name "' + name + '" is used ' +
                           str(all_names.count(name)) + ' times in the model. '
                           'All layer names should be unique.')
        
      
      
      self._nodes_by_depth = nodes_by_depth
      
      
      # return network_nodes, nodes_by_depth, layers, layers_by_depth


    def _run_internal_graph(self, inputs, training=None, mask=None):
        """Computes output tensors for new inputs.

        # Note:
            - Can be run on non-Keras tensors.

        Args:
            inputs: Tensor or nested structure of Tensors.
            training: Boolean learning phase.
            mask: (Optional) Tensor or nested structure of Tensors.

        Returns:
            output_tensors
        """
        print("_run_internal_graph --custom")
        inputs = self._flatten_to_reference_inputs(inputs)
        if mask is None:
          masks = [None] * len(inputs)
        else:
          masks = self._flatten_to_reference_inputs(mask)
        for input_t, mask in zip(inputs, masks):
          input_t._keras_mask = mask

        # Dictionary mapping reference tensors to computed tensors.
        tensor_dict = {}
        tensor_usage_count = self._tensor_usage_count
        for x, y in zip(self.inputs, inputs):
          y = self._conform_to_reference_input(y, ref_input=x)
          x_id = str(id(x))
          tensor_dict[x_id] = [y] * tensor_usage_count[x_id]

        nodes_by_depth = self._nodes_by_depth
        depth_keys = list(nodes_by_depth.keys())
        depth_keys.sort(reverse=True)
    
    
        for depth in depth_keys:
          nodes = nodes_by_depth[depth]
          for node in nodes:
            print(node.layer.name)
            if node.is_input:
              continue  # Input tensors already exist.

            if any(t_id not in tensor_dict for t_id in node.flat_input_ids):
              continue  # Node is not computable, try skipping.

            args, kwargs = node.map_arguments(tensor_dict)
            outputs = node.layer(*args, **kwargs)
            
                
            # Update tensor_dict.
            for x_id, y in zip(node.flat_output_ids, nest.flatten(outputs)):
              tensor_dict[x_id] = [y] * tensor_usage_count[x_id]
            
            
            if "branch_exit" in node.layer.name:
                print("branch exit activated")
                output_tensors = []
                for x_id, y in zip(node.flat_output_ids, nest.flatten(outputs)):
                    for x in self.outputs:
                      output_id = str(id(x))  
                      if output_id == x_id:
                        output_tensors.append(tensor_dict[x_id])
                      else:
                        # print(tensor_dict[x_id][0].shape)
                        output_tensors.append(tf.zeros(tensor_dict[x_id][0].shape))
                        
                      # x_id_output = str(id(x))
                      # assert x_id in tensor_dict, 'Could not compute output ' + str(x)
                      # output_tensors.append(tensor_dict[x_id])

                return nest.pack_sequence_as(self._nested_outputs, output_tensors)
            
            
        output_tensors = []
        for x in self.outputs:
          x_id = str(id(x))
          assert x_id in tensor_dict, 'Could not compute output ' + str(x)
          output_tensors.append(tensor_dict[x_id].pop())

        return nest.pack_sequence_as(self._nested_outputs, output_tensors)
#     def call(self, inputs, training=None, mask=None):
#         x = inputs
        
#         for layer in self.layers:
#             # print(layer)
#             x = layer(x)
            
#         return x

    def turnOnThresholds(self):
        for layers in self.layers:
            if hasattr(layers,'threshold'):
                layers.threshold = True

class Branch(tf.keras.Model):

  def __init__(self,name=''):
    super().__init__(name=name)
    self.layer = keras.layers.Layer(name=name)
    # self.layer.name = name
    # self.name=tf.compat.v1.get_default_graph().unique_name("branch")
    self.branchLayer1 = keras.layers.Flatten(name=tf.compat.v1.get_default_graph().unique_name("branch_flatten"))
    self.branchLayer2 = keras.layers.Dense(124, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch124"))
    self.branchLayer3 = keras.layers.Dense(64, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("branch64"))
    self.exit =keras.layers.Dense(10,name=tf.compat.v1.get_default_graph().unique_name("branch_exit"))

  def call(self, input_tensor, training=False):
    x = self.branchLayer1(input_tensor)
    x = tf.nn.relu(x)
    x = self.branchLayer2(x)
    x = tf.nn.relu(x)
    x = self.branchLayer3(x)
    x = self.exit(x)
    return x
                

inputs = keras.Input(shape=(227,227,3))
# targets = keras.Input(shape=(10,))
x = keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3))(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
x = keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
# ### first branch


branch_exit = Branch(tf.compat.v1.get_default_graph().unique_name("branch"))(x)
x = keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(4096, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(124, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("main_path124"))(x)
x = keras.layers.Dense(64, activation="relu",name=tf.compat.v1.get_default_graph().unique_name("main_path64"))(x)
x = keras.layers.Dense(10, activation="softmax", name=tf.compat.v1.get_default_graph().unique_name("exit"))(x)


# model = keras.Model(inputs=(inputs), outputs=[x,branch_exit], name="alexnet")
model = simpleModel(inputs=(inputs), outputs=[x,branch_exit], name="alexnet")

model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001,momentum=0.9), metrics=['accuracy'])
# student_model.save("models/alexNetv6_second_Exit.hdf5")
model.summary()

Model: "alexnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_30 (InputLayer)          [(None, 227, 227, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_130 (Conv2D)            (None, 55, 55, 96)   34944       ['input_30[0][0]']               
                                                                                                  
 batch_normalization_130 (Batch  (None, 55, 55, 96)  384         ['conv2d_130[0][0]']             
 Normalization)                                                                                   
                                                                                            

In [64]:
# model.call??
# model._run_internal_graph??
# model._map_graph_network(model.inputs,model.outputs, False)
# print("")
# print("reverse")
# print("")
model._map_graph_network(model.inputs,model.outputs, True)

_map_graph_network
branch_4  depth:  0
exit_24  depth:  0
main_path64_24  depth:  1
main_path124_24  depth:  2
dropout_24  depth:  3
dense_24  depth:  4
flatten_24  depth:  5
max_pooling2d_84  depth:  6
batch_normalization_134  depth:  7
conv2d_134  depth:  8
batch_normalization_133  depth:  9
conv2d_133  depth:  10
batch_normalization_132  depth:  11
conv2d_132  depth:  12
max_pooling2d_83  depth:  13
batch_normalization_131  depth:  14
conv2d_131  depth:  15
max_pooling2d_82  depth:  16
batch_normalization_130  depth:  17
conv2d_130  depth:  18
input_30  depth:  19

conv2d_130 :  18   input_30 :  19
batch_normalization_130 :  17   conv2d_130 :  18
max_pooling2d_82 :  16   batch_normalization_130 :  17
conv2d_131 :  15   max_pooling2d_82 :  16
batch_normalization_131 :  14   conv2d_131 :  15
max_pooling2d_83 :  13   batch_normalization_131 :  14
conv2d_132 :  12   max_pooling2d_83 :  13
batch_normalization_132 :  11   conv2d_132 :  12
conv2d_133 :  10   batch_normalization_132 :  11
b

In [55]:
# %%timeit
# print(test_ds.take(1).get_single_element())
x  = test_ds.take(1).get_single_element()
# for i in range(32):
result = model(x[0])
print(result)

_run_internal_graph --custom
input_26
conv2d_110
batch_normalization_110
max_pooling2d_70
conv2d_111
batch_normalization_111
max_pooling2d_71
branch_exit_22
branch exit activated
[<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>, [<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[  6.8509197,   3.2038624,   9.884068 ,  -2.0487866,  22.573503 ,
        -16.507559 , -23.67594  ,   3.1319938, -24.057451 ,  17.183311 ]],
      dtype=float32)>]]


In [21]:
for nodes in model._nodes_by_depth:
    print(dir(nodes))
    


['__abs__', '__add__', '__and__', '__bool__', '__ceil__', '__class__', '__delattr__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floor__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__le__', '__lshift__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rlshift__', '__rmod__', '__rmul__', '__ror__', '__round__', '__rpow__', '__rrshift__', '__rshift__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__trunc__', '__xor__', 'bit_length', 'conjugate', 'denominator', 'from_bytes', 'imag', 'numerator', 'real', 'to_bytes']
['__abs__', '__add__', '__and__', '__bool__', '__ceil__', '__class__', '__delattr__', '__dir__', '__d

In [5]:
# %%timeit
#using basic model with custom call function
# model.turnOnThresholds()
result = model.predict(test_ds.take(1))
print(result)
# predictions = []
# for i in result:
    # predictions.append(np.argmax(i,axis=1))
# print(predictions)

[[5.0970970e-20 2.4690530e-09 1.2282757e-02 2.9390292e-15 3.4506284e-05
  2.4547730e-13 3.0208079e-11 9.8768264e-01 8.1208469e-12 1.5698492e-07]]


In [62]:
%%timeit
## using no custom call functionality

# model.turnOnThresholds()
result = model.predict(test_ds.take(126))

380 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
%%timeit
## using custom call functionality

# model.turnOnThresholds()
result = model.predict(test_ds.take(126))

_run_internal_graph --custom
input_30
conv2d_130
batch_normalization_130
max_pooling2d_82
conv2d_131
batch_normalization_131
max_pooling2d_83
branch_4
conv2d_132
batch_normalization_132
conv2d_133
batch_normalization_133
conv2d_134
batch_normalization_134
max_pooling2d_84
flatten_24
dense_24
dropout_24
main_path124_24
main_path64_24
exit_24
385 ms ± 6.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
# print(test_ds.take(1).get_single_element())
x  = test_ds.take(1).get_single_element()
for i in range(126):
    result = model(x[0])
# print(result)

#### simpleModel with custom call function... surprisingly very very similar to the normal method...

394 ms ± 4.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
# print(test_ds.take(1).get_single_element())
x  = test_ds.take(1).get_single_element()
for i in range(126):
    result = model(x[0])
# print(result)

#### simpleModel with custom call function... surprisingly very very similar to the normal method...

431 ms ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
result = model.evaluate(test_ds.take(1))

1/1 [==============================] - 0s 376ms/step - loss: 2.3951 - accuracy: 0.0000e+00


In [41]:
result = model.evaluate(test_ds.take(1))

1/1 [==============================] - 0s 276ms/step - loss: 0.7777 - accuracy: 0.0000e+00
